In [1]:
import theano
from theano import tensor as T
import numpy as np
from theano.tensor.nnet.conv import conv2d

In [2]:
# generating some fake data labels
Y_ex = np.zeros((128, 6))

for i, r in enumerate(Y_ex):
    r[np.random.randint(6)] = 1
    Y_ex[i] = r

Y_ex.shape

(128, 6)

In [3]:
mbs = 128 # mini-batch size
n_channels, n_rows, n_cols = 1, 5, 21 # define protein "image" dimensions
n_conv = 256 # features out of convolution
h_depth = 50 # hidden layer depth
o_depth = 6 # output depth

In [4]:
X = theano.shared(np.array(xrange(mbs*n_channels*n_rows*n_cols), 
                           dtype=theano.config.floatX).reshape((mbs, n_channels, n_rows, n_cols)))

wc = theano.shared(np.array(xrange(n_conv*n_channels*n_rows*n_cols), 
                            dtype=theano.config.floatX)).reshape((n_conv, n_channels, n_rows, n_cols))

wh = theano.shared(np.array(xrange(n_conv*h_depth), dtype=theano.config.floatX)).reshape((n_conv, h_depth))

wh2 = theano.shared(np.array(xrange(h_depth*h_depth), dtype=theano.config.floatX)).reshape((h_depth, h_depth))

wy = theano.shared(np.array(xrange(h_depth*o_depth), dtype=theano.config.floatX)).reshape((h_depth, o_depth))

Y = theano.shared(Y_ex)

print 'X:', X.eval().shape
print 'Conv:', wc.eval().shape
convOut = conv2d(X, wc, border_mode='valid')  #border_mode={'valid', 'full'}

convOut = convOut.reshape((-1, n_conv))
print '-'*40
print 'ConvOut:', convOut.eval().shape
print 'wh1:', wh.eval().shape

hiddenOut1 = T.dot(convOut, wh)
print '-'*40
print 'Hidden1Out:', hiddenOut1.eval().shape
print 'wh2:', wh2.eval().shape
hiddenOut2 = T.dot(hiddenOut1, wh2)

print '-'*40
print 'Hidden2Out:', hiddenOut2.eval().shape

yOut = T.dot(hiddenOut2, wy)

print 'wy:', wy.eval().shape
print '-'*40
print 'yOut:', yOut.eval().shape
print '-'*40
smax_out = T.nnet.softmax(yOut)

test = smax_out.eval()
print 'ex:'
print test[:10]
print 'truth:'
print Y.eval()[:10]
print '-'*40
print 'preds:', test.shape
print 'true:', Y.eval().shape
print '-'*40
print 'mean:', T.mean(((Y.eval() - test)**2)).eval()
print 'cross:', T.mean(T.nnet.categorical_crossentropy(Y.eval(), test)).eval()

X: (128, 1, 5, 21)
Conv: (256, 1, 5, 21)
----------------------------------------
ConvOut: (128, 256)
wh1: (256, 50)
----------------------------------------
Hidden1Out: (128, 50)
wh2: (50, 50)
----------------------------------------
Hidden2Out: (128, 50)
wy: (50, 6)
----------------------------------------
yOut: (128, 6)
----------------------------------------
ex:
[[ 0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  1.]]
truth:
[[ 0.  0.  0.  0.  0.  1.]
 [ 1.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  1.  0.]]
----------------------------------------
preds: (128, 6)
true: (128, 6)
---------